In [18]:
import gc
import torch
import time

from pydantic import BaseModel, Field
from typing import List, Dict, Any, Tuple
from textwrap import dedent
from sentence_transformers import CrossEncoder

from langchain_community.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_compressors import FlashrankRerank
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.retrievers import BaseRetriever
from langchain_openai import ChatOpenAI
from langchain_classic.chains import RetrievalQA
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import CrossEncoderReranker

In [2]:
loader = PyPDFLoader('./data/투자설명서.pdf')

In [3]:
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'}, encode_kwargs={'batch_size':8})

In [4]:
docs = loader.load()
full_text = '\n\n'.join(doc.page_content for doc in docs)
text_splitter = SemanticChunker(embeddings=embeddings)
docs = text_splitter.create_documents([full_text])
for doc in docs:
    doc.metadata['source'] = '투자설명서.pdf'
print(len(docs))

243


In [5]:
gc.collect()
torch.cuda.empty_cache()
time.sleep(3)

In [6]:
faiss_store = FAISS.from_documents(docs, embedding=embeddings)
persist_dir = './data/faiss_index_dense'
faiss_store.save_local(persist_dir)

In [7]:
vectorstore = FAISS.load_local(persist_dir, embeddings=embeddings, allow_dangerous_deserialization=True)
vectorstore

In [8]:
gc.collect()
torch.cuda.empty_cache()
time.sleep(3)

In [9]:
class RelevanceScore(BaseModel):
    relevance_score: float = Field(description='문서가 쿼리와 얼마나 관련이 있는지를 나타내는 점수.')

In [10]:
def reranking_documents(query: str, docs: List[Document], top_n: int=3) -> List[Document]:
    parser = JsonOutputParser(pydantic_object=RelevanceScore)
    human_msg_prompt = PromptTemplate(
        template='''
1점부터 10점까지 점수를 매겨, 다음 문서가 질문과 얼마나 관련이 있는지 평가해주세요.
단순히 키워드가 일치하는 것이 아니라 쿼리의 구체적인 맥락과 의도를 고려하세요.
{format_instruction}
question: {query}
document: {doc}
relevance_score''',
        input_variables=['query', 'doc'],
        partial_variables={'format_instruction':parser.get_format_instructions()}
    )
    llm = ChatOpenAI(model='gpt-5-nano', temperature=0, max_completion_tokens=1024)
    chain = (
        human_msg_prompt
        | llm
        | parser
    )
    score_docs = []
    for doc in docs:
        input_data = {'query':query, 'doc':doc.page_content}
        try:
            score = float(chain.invoke(input_data)['relevance_score'])
        except Exception as e:
            print(f'오류 발생: {str(e)}')
            default_score = 5
            print(f'기본 점수 {default_score}점을 사용합니다.')
            score = default_score
        score_docs.append((doc, score))
    
    reranked_docs = sorted(score_docs, key=lambda x:x[1], reverse=True)

    return [doc for doc, _ in reranked_docs[:top_n]]

In [11]:
query = '이 회사의 2022년 영업이익 혹은 영업손실이 정확히 얼마야?'
initial_docs = vectorstore.similarity_search(query, k=5)
reranked_docs = reranking_documents(query, initial_docs)

오류 발생: Invalid json output: 
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 
기본 점수 5점을 사용합니다.
오류 발생: Invalid json output: 
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 
기본 점수 5점을 사용합니다.
오류 발생: Invalid json output: 
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 
기본 점수 5점을 사용합니다.


In [12]:
print(f'Query: {query}\n\n')
print('Top initial documents:')
for i, doc in enumerate(initial_docs):
    print(f'\nDocument {i+1}:')
    print(doc.page_content)
print('\n\nTop reranked documents:')
for i, doc in enumerate(reranked_docs):
    print(f'\nDocument {i+1}:')
    print(doc.page_content)

Query: 이 회사의 2022년 영업이익 혹은 영업손실이 정확히 얼마야?


Top initial documents:

Document 1:
회사위험
[가. 매출부진 및 지속적 손실 발생 위험]
당사는 매출이 부진한 가운데, 지속적으로 신약후보물질의 발굴과 보유 파
이프라인의 적응증 증가를 위하여 다양한 비임상 및 임상시험을 준비하고
있으며 이에 따라 관련비용의 지출이 꾸준히 발생하여 2021년 영업손실
130.1억원, 2022년 영업손실 149.1억원, 2023년 영업손실 122억원,
2024년 1분기 영업손실 24.2억원이 발생하였습니다. 또한 영업 외적 측면
에서도, 금융비용 등의 발생 영향으로 인해 2021년 당기순손실 130.7억원,
2022년 당기순손실 228.7억원, 2023년 당기순손실 116.1억원, 2024년
1분기 당기순손실 32.9억원이 발생하는 등 지속적인 적자 구조를 면
하지 못하고 있습니다.따라서 당사의 파이프라인에서 임상 성공을 통
한 기술이전, 상품화 성공 등의 성과를 이루어내지 못한다면 당사의 적자
구조를 개선하는 것은 불가능할 수 있으며, 자본력이 지속적으
로 감소하여 지속적인 연구개발비용 등의 지출이 필요한 당사
의 영업활동을 영위하는데 상당한 제약이 생길 수 있습니다. 또

한 (주)포베이커의 지분 취득 및 합병과 관련하여, 이는 단기적으로는 당
사의 기존 사업과는 무관한 분야의 신규사업 진출에 해당하며, 만
일 신규사업에 대한 성과를 내지 못하거나 사업적 시너지를 중
장기적으로 발생시키지 못할 경우 이는 당사 손익 내지 재무상
태에 오히려 악영향을 미칠 수 있습니다. 투자자 여러분들께서는 이 점
을 반드시 유의하여 주시기 바랍니다.

Document 2:
위 각 호에 관련된 부대사업 일체
다. 최근 3년간 요약재무정보 및 외부감사 여부

1) 최근 3년간 요약 재무상태표
(단위 : 원)
구분 2023년 2022년 2021년
 유동자산 976,148,260 1,751,083,731 1,809,632,798
 비유동자산 -

In [13]:
class CustomRetriever(BaseRetriever, BaseModel):
    vectorstore: Any = Field(description='Retrieval을 위한 벡터 저장소')

    class config:
        arbitrary_types_allowed = True
    
    def _get_relevant_documents(self, query: str, **kwargs) -> List[Document]:
        # reranking 후 반환할 문서 수
        num_docs = 2
        initial_docs = self.vectorstore.similarity_search(query, k=4)
        
        return reranking_documents(query, initial_docs, top_n=num_docs)

In [14]:
custom_retriever = CustomRetriever(vectorstore=vectorstore)
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=custom_retriever,
    return_source_documents=True
)
qa_chain.invoke(query)

오류 발생: Invalid json output: 
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 
기본 점수 5점을 사용합니다.
오류 발생: Invalid json output: 
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 
기본 점수 5점을 사용합니다.


{'query': '이 회사의 2022년 영업이익 혹은 영업손실이 정확히 얼마야?',
 'result': '2022년 영업손실은 149.1억원입니다.',
 'source_documents': [Document(id='f92c6bbc-5f09-47ae-9a55-77a4f6ff29b5', metadata={'source': '투자설명서.pdf'}, page_content='회사위험\n[가. 매출부진 및 지속적 손실 발생 위험]\n당사는 매출이 부진한 가운데, 지속적으로 신약후보물질의 발굴과 보유 파\n이프라인의 적응증 증가를 위하여 다양한 비임상 및 임상시험을 준비하고\n있으며 이에 따라 관련비용의 지출이 꾸준히 발생하여 2021년 영업손실\n130.1억원, 2022년 영업손실 149.1억원, 2023년 영업손실 122억원,\n2024년 1분기 영업손실 24.2억원이 발생하였습니다. 또한 영업 외적 측면\n에서도, 금융비용 등의 발생 영향으로 인해 2021년 당기순손실 130.7억원,\n2022년 당기순손실 228.7억원, 2023년 당기순손실 116.1억원, 2024년\n1분기 당기순손실 32.9억원이 발생하는 등 지속적인 적자 구조를 면\n하지 못하고 있습니다.따라서 당사의 파이프라인에서 임상 성공을 통\n한 기술이전, 상품화 성공 등의 성과를 이루어내지 못한다면 당사의 적자\n구조를 개선하는 것은 불가능할 수 있으며, 자본력이 지속적으\n로 감소하여 지속적인 연구개발비용 등의 지출이 필요한 당사\n의 영업활동을 영위하는데 상당한 제약이 생길 수 있습니다. 또\n\n한 (주)포베이커의 지분 취득 및 합병과 관련하여, 이는 단기적으로는 당\n사의 기존 사업과는 무관한 분야의 신규사업 진출에 해당하며, 만\n일 신규사업에 대한 성과를 내지 못하거나 사업적 시너지를 중\n장기적으로 발생시키지 못할 경우 이는 당사 손익 내지 재무상\n태에 오히려 악영향을 미칠 수 있습니다. 투자자 여러분들께서는 이 점\n을 반드시 유의하여 주시기 바랍니다.'),
  Document(

In [15]:
# 2. Reranker 모델 초기화
reranker = FlashrankRerank()

# 3. 기본 Retriever 설정 (Vectorstore)
# 초기 검색(k) 결과 수를 10개로 늘려 Reranker가 더 많은 후보 중에서 순위를 재산정하도록 합니다.
base_retriever = vectorstore.as_retriever(search_kwargs={'k': 10})

# 4. ContextualCompressionRetriever 설정
# base_retriever가 반환한 문서를 reranker가 재정렬하고 관련 없는 문서는 필터링합니다.
compression_retriever = ContextualCompressionRetriever(
    base_compressor=reranker, 
    base_retriever=base_retriever
)

# 5. RetrievalQA 체인 재구성
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
qa_chain_rerank = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=compression_retriever, # 압축/재정렬 Retriever 사용
    return_source_documents=True
)

# 6. 체인 실행 및 결과 확인
query = '이 회사의 2022년 영업이익 혹은 영업손실이 정확히 얼마야?'
result = qa_chain_rerank.invoke(query)

# 결과 출력
print(f"[답변]: {result['result']}")
print("\n--- [참조된 문서]: ---")
for doc in result['source_documents']:
    print(doc.page_content)
    print("-" * 20)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[답변]: 2022년 영업손실은 149억 원입니다.

--- [참조된 문서]: ---
다. 다양한 종류의 주식 
 
당사는 본 보고서 작성기준일 현재 해당 사항이 없습니다. 5. 정관에 관한 사항 
 
가. 정관 변경 이력 
Ⅰ. 발행할 주식의 총수 75,000,000 25,000,000 100,000,000 -
Ⅱ. 현재까지 발행한 주식의 총수 13,602,977 - 13,602,977 -
Ⅲ. 현재까지 감소한 주식의 총수 - - - -
1. 감자 - - - -
2. 이익소각 - - - -
3. 상환주식의 상
환 - - - -
4. 기타 - - - -
Ⅳ. 발행주식의 총수 (Ⅱ-Ⅲ) 13,602,977 - 13,602,977 -
Ⅴ. 자기주식수 - - - -
Ⅵ. 유통주식수 (Ⅳ-Ⅴ) 13,602,977 - 13,602,977 -
정관변
경일 해당주총명 주요변경사항 변경이유
2020.03
.25
제14기 정기주
주총회
제2조(목적)
제17조(전환사채의 발
행)
제18조(신주인수권부사
채의 발행)
제19조(이익참가부사채
의 발행)
제20조(교환사채의 발
행)
- 관련 사업 확장가능성에 따른 신규 목적사업
추가
- 한도 증액
- 한도 증액
- 한도 증액
- 한도 증액
2022.03
.23
제16기 정기주
주총회
제2조(목적)
제8조의 2(주식 등의 전
자등록)
제47조(감사의 선임)
- 관련 사업 확장가능성에 따른 신규 목적사업
추가
- 전자증권제도 도입 반영
- 감사 선임에 관한 조문 정비 및 전자투표 도입
시 결의 요건 내용 반영

나. 사업 목적 현황 
  
 
다. 사업 목적 변경   
 
1. 사업목적 변경 내용 
2024.03
.28
제18기 정기주
주총회
제2조(목적)
제10조(신주인수권)
제39조의2(이사의 책임
감경)
- 신사업 진행을 위한 사업 목적 추가
- 긴급한 자금조달을 위해 대상자 범위 확대를
위한 조문 정비
- 표준정관에 따른 조항 신설
구 분 사업목적 사업영위 여부
1 면역학 및 분자생물학 기법을 이용한 신약의 연구개발 및 제조, 판

In [16]:
gc.collect()
torch.cuda.empty_cache()
time.sleep(3)

In [19]:
cross_encoder = HuggingFaceCrossEncoder(model_name='Dongjin-kr/ko-reranker', model_kwargs={'device':'cuda'})
reranker = CrossEncoderReranker(model=cross_encoder, top_n=3)
base_retriever = vectorstore.as_retriever()

config.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [20]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=base_retriever
)
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

In [21]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=compression_retriever,
    return_source_documents=True
)

In [22]:
query = '이 회사의 2022년 영업이익 혹은 영업손실이 정확히 얼마야?'
result = qa_chain.invoke(query)

# 결과 출력
print(f"[답변]: {result['result']}")
print("\n--- [참조된 문서]: ---")
for doc in result['source_documents']:
    print(doc.page_content)
    print("-" * 20)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[답변]: 2022년 영업손실은 149.1억원입니다.

--- [참조된 문서]: ---
회사위험
[가. 매출부진 및 지속적 손실 발생 위험]
당사는 매출이 부진한 가운데, 지속적으로 신약후보물질의 발굴과 보유 파
이프라인의 적응증 증가를 위하여 다양한 비임상 및 임상시험을 준비하고
있으며 이에 따라 관련비용의 지출이 꾸준히 발생하여 2021년 영업손실
130.1억원, 2022년 영업손실 149.1억원, 2023년 영업손실 122억원,
2024년 1분기 영업손실 24.2억원이 발생하였습니다. 또한 영업 외적 측면
에서도, 금융비용 등의 발생 영향으로 인해 2021년 당기순손실 130.7억원,
2022년 당기순손실 228.7억원, 2023년 당기순손실 116.1억원, 2024년
1분기 당기순손실 32.9억원이 발생하는 등 지속적인 적자 구조를 면
하지 못하고 있습니다.따라서 당사의 파이프라인에서 임상 성공을 통
한 기술이전, 상품화 성공 등의 성과를 이루어내지 못한다면 당사의 적자
구조를 개선하는 것은 불가능할 수 있으며, 자본력이 지속적으
로 감소하여 지속적인 연구개발비용 등의 지출이 필요한 당사
의 영업활동을 영위하는데 상당한 제약이 생길 수 있습니다. 또

한 (주)포베이커의 지분 취득 및 합병과 관련하여, 이는 단기적으로는 당
사의 기존 사업과는 무관한 분야의 신규사업 진출에 해당하며, 만
일 신규사업에 대한 성과를 내지 못하거나 사업적 시너지를 중
장기적으로 발생시키지 못할 경우 이는 당사 손익 내지 재무상
태에 오히려 악영향을 미칠 수 있습니다. 투자자 여러분들께서는 이 점
을 반드시 유의하여 주시기 바랍니다.
--------------------
위 각 호에 관련된 부대사업 일체
다. 최근 3년간 요약재무정보 및 외부감사 여부

1) 최근 3년간 요약 재무상태표
(단위 : 원)
구분 2023년 2022년 2021년
 유동자산 976,148,260 1,751,083,731 1,809,632,798
 비유동자산 - - -
자산총계 976,148,260 1

In [23]:
gc.collect()
torch.cuda.empty_cache()
time.sleep(3)